In [ ]:
import os
import yaml
from itertools import product
import math

In [ ]:
def map_vars_to_config():

    var_config_mapping = {
        'data': {
            'nfcorpus': {'data': {
                'test_dat_key': 'test_nfcorpus_value',
            }}
        },
        'device': {
            'gpu': {
                'inference_device': 'gpu',
                'tensor_ops_device': 'gpu',
                },
            'cpu': {
                'inference_device': 'cpu',
                'tensor_ops_device': 'cpu',}
        }
    }
    return var_config_mapping

In [1]:
def deep_update(original, update):
    for key, value in update.items():
        if isinstance(value, dict):
            original[key] = deep_update(original.get(key, {}), value)
        else:
            original[key] = value
    return original


def save_config(config, base_dir, experiment_name):
    full_path = os.path.join(base_dir, experiment_name)
    os.makedirs(full_path, exist_ok=True)
    with open(os.path.join(full_path, "config.yaml"), 'w') as f:
        yaml.dump(config, f)

    #EMNLP EVAL CONFIG SAVING
    # measures = []
    # for i in range(1, 176):
    #     measures.append(f"ndcg_cut_{i}")
    # for i in range(1, 176):
    #     measures.append(f"map_cut_{i}")
    # for i in range(1, 176):
    #     measures.append(f"P_{i}")
    # for i in range(1, 176):
    #     measures.append(f"recall_{i}")

    # eval_params = {
    #     'measures': measures,
    #     'qrels_path' : os.path.join(os.path.dirname(config['data']['run_path']),'qrels.txt'),
    #     'logging': {'level': 'DEBUG'},
    # }

    # with open(os.path.join(full_path, "eval_config.yaml"), 'w') as f:
    #     yaml.dump(eval_params, f)

In [ ]:
EXP_DIR = 'trials/test_param_writer'
if not os.path.exists(EXP_DIR):
    os.makedirs(EXP_DIR)

BASE_CONFIG_PATH = '../configs/base_config.yaml'

In [ ]:
#set the parmeters
#options/explanations of params:
'''
param_grid = {
}
'''

# Define the parameters
param_grid = {
    'data': ['nfcorpus'],
    'device': ['gpu', 'cpu'],
}



In [ ]:
# Load the base config file
with open(BASE_CONFIG_PATH, 'r') as f:
    base_config = yaml.safe_load(f)

#for file naming, don't use these param keys:
PARAM_NAMES_TO_OMMIT = {'device'} #

# Generate and save config files for each combination
for param_values in product(*param_grid.values()):
    param_values_dict = dict(zip(param_grid.keys(), param_values))
    experiment_name = '_'.join([f"{value}" for param, value in param_values_dict.items() if param not in PARAM_NAMES_TO_OMMIT])
    updated_config = yaml.safe_load(yaml.dump(base_config))  # deep copy

    # Apply updates to the config based on the current parameter values
    var_config_mapping = map_vars_to_config(
        #dimensionality can be set this way... for gemini
    )
    for param, value in param_values_dict.items():
        if param in var_config_mapping and value in var_config_mapping[param]:
            deep_update(updated_config, var_config_mapping[param][value])
        else:
            print(f"No mapping found for parameter '{param}' with value '{value}'")

    save_config(updated_config, EXP_DIR, experiment_name, pw = pw)
